## Import - Libraries

In [1]:
import os
import calendar
import numpy as np
import pandas as pd

## Import - Data Files & Additional Files

In [2]:
os.chdir(r'C:\Users\bhava\Downloads\Re__Capstone_Project__Preliminary_Discussion\Reviews\Raw Files')

In [3]:
raw_df= pd.read_csv(r'combinedReviews.csv')

In [4]:
major_locations_df=pd.read_csv(r'C:\Users\bhava\Downloads\Re__Capstone_Project__Preliminary_Discussion\Reviews\Mapping Files\Major Locations.csv')
locations_mapping_df=pd.read_csv(r'C:\Users\bhava\Downloads\Re__Capstone_Project__Preliminary_Discussion\Reviews\Mapping Files\Location Mapping.csv')

## Functions and Dictionaries

In [5]:
def clean_jobFunction(jobFunction):
    try:
        clean_jobFunc=jobFunction.split(' for ')[0]
        return clean_jobFunc
    except:
        return jobFunction

In [6]:
def extract_experience(jobFunction):
    try:
        exp=jobFunction.split(' for ')[1].split(' in ')[0].split(' ')[0]
        return exp
    except:
        return -1

In [7]:
def extract_location(jobFunction):
    try:
        loc=jobFunction.split(' for ')[1].split(' in ')[1]
        return loc
    except:
        return 'Not Specified'

In [8]:
def clean_location(location_name):
    for loc in major_locations:
        ind=-1
        if location_name.find(loc)!=-1:
            ind=location_name.find(loc)
            return loc
    if ind==-1:
        return location_name
    

In [9]:
def extract_exp_unit(jobFunction):
    try:
        if jobFunction.split(' for ')[1].split(' in ')[0].find('year')!=-1:
            return 12
        elif jobFunction.split(' for ')[1].split(' in ')[0].find('month')!=-1:
            return 1
    except:
        return 1

In [10]:
#List of Major Locations
major_locations=major_locations_df['Major Locations'].to_list()
#Dictionary of Locations to be mapped
location_mapping=locations_mapping_df.set_index('Original Value')['Mapped Value'].to_dict()

## Exploratory Data Analysis

In [11]:
print('Volume of Data :',raw_df.shape)

Volume of Data : (97097, 16)


In [12]:
print('Column Data Types:')
raw_df.dtypes

Column Data Types:


skillDevelopment    float64
reviewText           object
location             object
cons                 object
salaryBenefits      float64
workLifeBalance     float64
pros                 object
workSatisfaction    float64
postedon             object
jobSecurity         float64
careerGrowth        float64
companyCulture      float64
company              object
jobFunction          object
overallScore        float64
file                 object
dtype: object

In [13]:
print('Number of Nulls in each column:')
for col in raw_df.columns:
    print(col,' :',raw_df[col].isna().sum())

Number of Nulls in each column:
skillDevelopment  : 91
reviewText  : 53875
location  : 30584
cons  : 11166
salaryBenefits  : 646
workLifeBalance  : 89
pros  : 6667
workSatisfaction  : 643
postedon  : 1
jobSecurity  : 633
careerGrowth  : 651
companyCulture  : 624
company  : 1
jobFunction  : 200
overallScore  : 0
file  : 0


## Data Cleaning and Pre-Processing

### Locations

In [14]:
#Extracting Expirience and Location from jobFunction column
raw_df['extract_location']=raw_df['jobFunction'].apply(lambda x:extract_location(x))

In [15]:
#Imputing Blanks in Location column with data extracted from jobFunction, if available
raw_df['cleaned_location']=np.where(raw_df['location'].isna(),raw_df['extract_location'],raw_df['location'])
#Imputing unknown locations with Not Specified
raw_df['cleaned_location']=raw_df['cleaned_location'].fillna('Not Specified')

In [16]:
#Extracting location city from location columns
raw_df['cleaned_location']=raw_df['cleaned_location'].apply(lambda x:clean_location(str(x)))
#Harmonizing Locations
raw_df['cleaned_location']=raw_df['cleaned_location'].map(location_mapping)

In [17]:
#Bucketing all last 10% of the locations into 'Others'
major_locations.append('Not Specified')
raw_df['cleaned_location']=np.where(raw_df['cleaned_location'].isin(major_locations),raw_df['cleaned_location'],'Others')

## Experience

In [18]:
raw_df['clean_jobFunc']=raw_df['jobFunction'].apply(lambda x:clean_jobFunction(x))
raw_df['experience']=raw_df['jobFunction'].apply(lambda x:extract_experience(x))

In [19]:
raw_df['experience_unit']=raw_df['jobFunction'].apply(lambda x:extract_exp_unit(x))


## Output

In [20]:
#raw_df=raw_df.drop(columns={'experience_unit'})

In [21]:
raw_df.to_csv(r'C:\Users\bhava\Downloads\Re__Capstone_Project__Preliminary_Discussion\Reviews\Intermediate Files\cleaned_location.csv',index=False)